In [1]:
# Set up the environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [3]:
# Upload the data
training = pd.read_csv('data/processed/training.csv', index_col=0)

y_train = training['CURROPER']
X_train = training.drop('CURROPER', axis=1)

testing = pd.read_csv('data/processed/training.csv', index_col=0)
y_test = testing['CURROPER']
X_test = testing.drop('CURROPER', axis=1)

# X_train = pd.read_csv('data/processed/X_train.csv', index_col=0)
# X_test = pd.read_csv('data/processed/X_test.csv', index_col=0)
# y_train = pd.read_csv('data/processed/y_train.csv', index_col=0)
# y_test = pd.read_csv('data/processed/y_test.csv', index_col=0)

In [10]:
training.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55025 entries, 2905 to 78034
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NUMBRANCH          55025 non-null  float64
 1   UGDS               55025 non-null  float64
 2   TUITFTE            55025 non-null  float64
 3   INEXPFTE           55025 non-null  float64
 4   PFTFAC             55025 non-null  float64
 5   UG25abv            55025 non-null  float64
 6   COMP_ORIG_YR4_RT   55025 non-null  float64
 7   WDRAW_ORIG_YR4_RT  55025 non-null  float64
 8   ENRL_ORIG_YR4_RT   55025 non-null  float64
 9   DEBT_MDN           55025 non-null  float64
 10  Year               55025 non-null  float64
 11  Cost               55025 non-null  float64
 12  Complete           55025 non-null  float64
 13  RetentionFT        55025 non-null  float64
 14  PREDDEG_1          55025 non-null  float64
 15  PREDDEG_2          55025 non-null  float64
 16  PREDDEG_3          

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55025 entries, 2905 to 78034
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   NUMBRANCH          55025 non-null  float64
 1   UGDS               55025 non-null  float64
 2   TUITFTE            55025 non-null  float64
 3   INEXPFTE           55025 non-null  float64
 4   PFTFAC             55025 non-null  float64
 5   UG25abv            55025 non-null  float64
 6   COMP_ORIG_YR4_RT   55025 non-null  float64
 7   WDRAW_ORIG_YR4_RT  55025 non-null  float64
 8   ENRL_ORIG_YR4_RT   55025 non-null  float64
 9   DEBT_MDN           55025 non-null  float64
 10  Year               55025 non-null  float64
 11  Cost               55025 non-null  float64
 12  Complete           55025 non-null  float64
 13  RetentionFT        55025 non-null  float64
 14  PREDDEG_1          55025 non-null  float64
 15  PREDDEG_2          55025 non-null  float64
 16  PREDDEG_3          

In [7]:
y_train.head()

2905      True
27743      NaN
60310     True
68067    False
15350      NaN
Name: CURROPER, dtype: object

In [ ]:
y = y_train.CURROPER.ravel()

# Modeling
## Logistic Regression

Hyperparameters to tune:
- C - inverse of regularization strength; positive float; smaller values are stronger regularization, may lead to underfit model; large C may lead to overfitting
- penalty (l1, l2, elasticnet, none) 
- l1_ratio - for elastic-net paramter mixing: l1_ratio = 0 == L2 penalty; l1_ratio = 1 == L1 penalty, so no need to use l1 and l2 as penalty parameters, since they will be encompassed in the elastic net values

In [ ]:
# Import and instantiate model
from sklearn.linear_model import LogisticRegression

In [ ]:
# Hyperparameter search
from sklearn.model_selection import RandomizedSearchCV

c_grid = [0.001, 0.01, 0.1, 1, 10]
pen_grid = ['elasticnet']
l1_ratio_grid = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
max_iter_grid = [100, 500, 1000, 1500, 2000]

lr_grid = {'C':c_grid, 'penalty':pen_grid, 'l1_ratio':l1_ratio_grid,'max_iter':max_iter_grid}

Note: trying to use pen_grid = ['none', 'elasticnet'] led to errors, since the l1_ratio parameter is only valid for elastic net penatly, not none. If I want to train a model with no penalty, I will have to run a separate Grid search.

In [ ]:
logreg = LogisticRegression(solver = 'saga')
logreg_cv = RandomizedSearchCV(logreg, lr_grid, cv=5)

Note - default solver 'lbfgs' can't handle elasticnet penalty.
Note - with default max_iter = 100, kept getting ConvergenceWarning: The max_iter was reached which means the coef_ did not converge, so I added max_iter as a grid search parameter

In [ ]:
start= time.time()
logreg_cv.fit(X_train, y)
end = time.time()
print('GridSearch Time:', end-start)

In [ ]:
print("Best params: " , logreg_cv.best_params_)
print("Best score: ", logreg_cv.best_score_)

### Grid Results
- Original run: l1_ratio=0.25, max_iter=2000, C=0.001
- Second run: l1_ratio=1, max_iter= 2000, C=0.001
- Third run: l1_ratio = 0.1, max_iter = 2000, C=0.01
- Another run: l1_raio = 0.9, max_iter = 500, C=0.0001

### Run the model with the best parameters

In [ ]:
modelLR = LogisticRegression(C = 0.01, penalty = 'elasticnet', l1_ratio=0.1, max_iter = 2000, solver = 'saga')
start = time.time()
modelLR.fit(X_train, y)
end = time.time()
print("Fit time = ", end - start)

start = time.time()
lr_pred = modelLR.predict(X_test)
end = time.time()
print("Predict time = ", end - start)

lr_pred_prob = modelLR.predict_proba(X_test)[:, 1]

### Evaluate the model

In [ ]:
# Evaluate model
# Confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, lr_pred))

In [ ]:
# Classification report
print(classification_report(y_test, lr_pred))

In [ ]:
# ROC curve
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_test, lr_pred_prob)

plt.plot( [0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label='Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression ROC Curve')
plt.show()

In [ ]:
print("AUC: ", roc_auc_score(y_test, lr_pred_prob))

In [ ]:
# Coefficients
coeffic = modelLR.coef_
coeffic = coeffic[0]
labels = [i for i in X_train.columns]
numLab = len(labels)

In [ ]:
coeffic = pd.DataFrame(coeffic, index=labels)
coeffic.columns= ['Coefficient']
coeffic.sort_values(by='Coefficient', inplace=True, ascending=False)
coeffic.head()

In [ ]:
coeffic.plot.bar(y='Coefficient')
#plt.xticks(range(0, numLab), labels, rotation='vertical')
plt.show()

### Coefficient comments
- Enrollment seems to be the most important predictor. [But it's worth checking how many values were imputed]
- Year probably shouldn't be a predictor variable - it was included to distinguish the same schools across different years, to see at which point schools closed... (which I didn't actually explore)
- Preddeg3 = Schools that offer predominantly bachelor's degrees (I think). I think this was the largest group of schools, so it may just be a data balance size issue. (May be predominantly graduate degrees)

- Negative coefficient values
-- Control3 = Private for profit school
-- Control2 = Private nonprofit
--(So does this mean that public schools have no effect, or positive coeff value?)
- Withdraw
- UG25abv 

## Decision Tree
Parameters to search:
- max_features
- max_depth
- min_samples_leaf

In [ ]:
from sklearn.tree import DecisionTreeClassifier
#from sklearn.model_selection import RandomizedSearchCV

In [ ]:
criterion_grid = ['gini', 'entropy']
max_depth_grid = ['None', 3, 5, 10, 20, 50, 75, 100]
min_sample_split_grid = [2, 5, 10, 25, 50, 75, 100]
min_samples_leaf_grid = [2, 5, 10, 25, 50]
feature_grid = ["auto", "log2", 4, 5, 10, 12, 15, 20]

dtc_grid = {'criterion': criterion_grid, 
              'max_depth':max_depth_grid,
              'min_samples_split':min_sample_split_grid,
              'min_samples_leaf':min_samples_leaf_grid,
              'max_features':feature_grid }

In [ ]:
tree = DecisionTreeClassifier()
tree_cv = RandomizedSearchCV(tree, dtc_grid, cv=5)
tree_cv.fit(X_train, y_train)

In [ ]:
print("Best params: " , tree_cv.best_params_)
print("Best score: ", tree_cv.best_score_)

### Grid Search Results
- First run: min_sample_split=2; min_samples_leaf=25; max_features=20, max_depth=20, criterion=entropy
- Second run: min_sample_split=75, min_samples_leaf=25; max_features=20, max_depth=20, criterion='entropy'
- Another run: min_sample_split=100; min_samples_leaf=25; max_features=15; max_depth=50; criterion = entropy

### Explore just the criterion and max_depth features, leaving the other arguments as the default. Plot the accuracy of different tree depths using both criterion measures.
Code modified from https://towardsdatascience.com/decision-tree-build-prune-and-visualize-it-using-python-12ceee9af752

In [ ]:
from sklearn import metrics
max_depth = []
acc_gini = []
acc_entropy = []
for i in range(1, 31):
    gtree = DecisionTreeClassifier(criterion='gini', max_depth=i)
    gtree.fit(X_train, y_train)
    gpredict = gtree.predict(X_test)
    acc_gini.append(metrics.accuracy_score(y_test, gpredict))
    ##
    etree = DecisionTreeClassifier(criterion='entropy', max_depth=i)
    etree.fit(X_train, y_train)
    epredict = etree.predict(X_test)
    acc_entropy.append(metrics.accuracy_score(y_test, epredict))
    ##
    max_depth.append(i)

In [ ]:
trees = pd.DataFrame({'acc_gini':pd.Series(acc_gini),
                     'acc_entropy':pd.Series(acc_entropy),
                     'max_depth':pd.Series(max_depth)})

plt.plot('max_depth', 'acc_gini', data=trees, label='gini')
plt.plot('max_depth', 'acc_entropy', data=trees, label='entropy')
#plt.vlines(12, 0.855, 0.880)
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# Train the decision tree using the identified hyperparameters
modelDT = DecisionTreeClassifier(criterion='gini', max_depth=12)
modelDT.fit(X_train, y_train)

In [ ]:
#from sklearn.tree import plot_tree
#plot_tree(modelDT)
# Code from https://towardsdatascience.com/decision-tree-build-prune-and-visualize-it-using-python-12ceee9af752

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus

dot_data = StringIO()
export_graphviz(modelDT, out_file=dot_data, filled=True, feature_names=X_train.columns)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('tree.png')
Image(graph.create_png())

In [ ]:
from sklearn.tree import plot_tree
plot_tree(modelDT)

In [ ]:
# Use the trained tree to predict the testing data
dt_pred = modelDT.predict(X_test)
dt_pred_prob = modelDT.predict(X_test)

In [ ]:
# Run this block for model evaluation 
#from sklearn import metrics
print("Model Metrics")
print("Accuracy:", metrics.accuracy_score(y_test, dt_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test, dt_pred))
print('Precision score for "Yes"' , metrics.precision_score(y_test, dt_pred, pos_label = 1))
print('Recall score for "No"' , metrics.recall_score(y_test, dt_pred, pos_label = 0))


In [ ]:
print(confusion_matrix(y_test, dt_pred))

In [ ]:
print(classification_report(y_test, dt_pred))

## Random Forest Classifier

Hyperparameters:
- number of features per tree
- number of trees per forest (n_estimators)
- depth(?)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
n_tree_grid = [10,50,100,200,250,500]
crit_grid = ['gini', 'entropy']
max_depth_grid = ['None', 3, 5, 10, 15, 20]
min_sample_split_grid = [2, 5, 10, 20]
min_samples_leaf_grid = [2, 5, 10, 25, 50]
feature_grid = ["auto", "log2", 4, 5, 10, 12, 15, 20]

rfc_grid = {'n_estimators':n_tree_grid,
              'criterion': crit_grid, 
              'max_depth':max_depth_grid,
              'min_samples_split':min_sample_split_grid,
              'min_samples_leaf':min_samples_leaf_grid,
              'max_features':feature_grid }

In [ ]:
forest = RandomForestClassifier()
start = time.time()
forest_cv = RandomizedSearchCV(forest, rfc_grid, cv=5)
forest_cv.fit(X_train, y)
end = time.time()
print("GridSearch time: ", end-start)

In [ ]:
print("Best params: " , forest_cv.best_params_)
print("Best score: ", forest_cv.best_score_)

### Grid Search Results
- First run: n_estimators=200; min_sample_split=2; min_samples_leaf=2, max_features=10; max_depth=15; criterion='gini'
- Second run: n_estimators=50; min_sample_split=5; min_samples_leaf=25; max_features=10; max_depth=10; criterion='gini'
- Another run: n_estimators=10, split=10, leaf=5, features=12, depth=20, crit=entropy

In [ ]:
# Choose default for split, leaf; use 'auto' for max features, which will default to sqrt(n_feature), 
# max_depth is half of depth of single tree (above)
modelRF = RandomForestClassifier(n_estimators = 100, max_features='auto',
                                max_depth=6, criterion='gini')
modelRF.fit(X_train, y)

In [ ]:
rf_pred = modelRF.predict(X_test)
rf_pred_prob = modelRF.predict_proba(X_test)

In [ ]:
print("Random Forest Classifier model")
print("Accuracy:", metrics.accuracy_score(y_test, rf_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test, rf_pred))
print('Precision score for Yes' , metrics.precision_score(y_test, rf_pred, pos_label = 1))
print('Recall score for No' , metrics.recall_score(y_test, rf_pred, pos_label = 0))

In [ ]:
print(confusion_matrix(y_test, rf_pred))

In [ ]:
print(classification_report(y_test, rf_pred))